In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import optuna
from lightgbm.callback import early_stopping


In [ ]:
# RMSPE 계산 함수
def rmspe(y_true, y_pred):
    return  np.sqrt(np.mean(np.square((y_true - y_pred) / (y_true))))

# LightGBM에서 사용할 수 있는 형태의 RMSPE 평가 함수
def rmspe_lgb(y_pred, data):
    y_true = data.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
df = pd.read_csv('./btc_selected_df_1.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.shape

(66696, 129)

# optuna를 활용한 파라미터 optimization

In [ ]:
X = df.drop('dv5_realized_volatility_mean0', axis =1)
y = df['dv5_realized_volatility_mean0']

In [ ]:
# 최적화할 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 범위 설정
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
    }

    # K-Fold 교차 검증
    kf = KFold(n_splits=5)
    rmspes = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle = False)

        # 모델 훈련
        gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train), 500,
                        valid_sets = lgb.Dataset(X_valid, label=y_valid),
                        callbacks=[early_stopping(stopping_rounds=20)])

        # 예측 및 평가
        preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        rmspe_value = rmspe(y_test, preds)
        rmspes.append(rmspe_value)


    avg_rmspe = np.mean(rmspes)

    return avg_rmspe



In [ ]:
# Optuna를 사용한 하이퍼파라미터 최적화
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 출력
print('Best trial:', study.best_trial.params)
print(study.best_trial.value)

[I 2023-10-28 14:32:54,958] A new study created in memory with name: no-name-eb3963f9-4ffc-4787-85fc-d0b0ee9d1f79
<ipython-input-45-3d6f33735477>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-45-3d6f33735477>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
<ipython-input-45-3d6f33735477>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('featu

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[123]	valid_0's rmse: 4.98108e-05
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[100]	valid_0's rmse: 4.97728e-05
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[137]	valid_0's rmse: 4.96869e-05
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[130]	valid_0's rmse: 4.96786e-05
Training until validation scores don't improve for 20 rounds


[I 2023-10-28 14:33:27,295] Trial 0 finished with value: 0.4440975026843743 and parameters: {'lambda_l1': 0.0007218625245839275, 'lambda_l2': 0.19261774136743276, 'num_leaves': 45, 'feature_fraction': 0.9458766357182261, 'bagging_fraction': 0.8206406119744858, 'bagging_freq': 5, 'min_child_samples': 61, 'learning_rate': 0.03356754299216251, 'max_depth': 10}. Best is trial 0 with value: 0.4440975026843743.


Early stopping, best iteration is:
[165]	valid_0's rmse: 5.56689e-05


<ipython-input-45-3d6f33735477>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-45-3d6f33735477>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
<ipython-input-45-3d6f33735477>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-45-3d6f33735477>:13: FutureWarning: suggest_uniform has been deprecated in

...

Early stopping, best iteration is:
[104]	valid_0's rmse: 5.62374e-05


<ipython-input-45-3d6f33735477>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-45-3d6f33735477>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
<ipython-input-45-3d6f33735477>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-45-3d6f33735477>:13: FutureWarning: suggest_uniform has been deprecated in

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[110]	valid_0's rmse: 4.95298e-05
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[114]	valid_0's rmse: 4.94998e-05
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[125]	valid_0's rmse: 4.94309e-05
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[91]	valid_0's rmse: 4.99067e-05
Training until validation scores don't improve for 20 rounds


[I 2023-10-28 15:14:20,356] Trial 99 finished with value: 0.4484832765570238 and parameters: {'lambda_l1': 0.008259957735146383, 'lambda_l2': 0.0003585775351971678, 'num_leaves': 51, 'feature_fraction': 0.8127726569243039, 'bagging_fraction': 0.8765551704869088, 'bagging_freq': 4, 'min_child_samples': 44, 'learning_rate': 0.0741582215921919, 'max_depth': 14}. Best is trial 90 with value: 0.4361816481224648.


Early stopping, best iteration is:
[96]	valid_0's rmse: 5.59857e-05
Best trial: {'lambda_l1': 0.0032551250961011156, 'lambda_l2': 0.00010150845162836709, 'num_leaves': 56, 'feature_fraction': 0.8036517266350524, 'bagging_fraction': 0.8576909504868375, 'bagging_freq': 4, 'min_child_samples': 40, 'learning_rate': 0.10637968859384557, 'max_depth': 15}
0.4361816481224648


In [ ]:
best_params = {
    'lambda_l1': 0.003,
    'lambda_l2': 0.001,
    'num_leaves': 56,
    'feature_fraction': 0.80,
    'bagging_fraction': 0.86,
    'bagging_freq': 4,
    'min_child_samples': 40,
    'learning_rate': 0.1,
    'max_depth': 15
}


In [ ]:
X_train = df[:50000].drop('dv5_realized_volatility_mean0', axis = 1)
y_train = df[:50000]['dv5_realized_volatility_mean0']

X_test = df[50000:].drop('dv5_realized_volatility_mean0', axis = 1)
y_test = df[50000:]['dv5_realized_volatility_mean0']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle = False)

# # LightGBM 데이터셋으로 변환
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [ ]:
print('Starting training...')
gbm = lgb.train(best_params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                callbacks=[early_stopping(stopping_rounds=20)]
                )

# 모델 예측
print('Starting predicting...')
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# y_test = abs(y_test)
# y_pred = abs(y_pred)

# 모델 평가
print('The RMSPE of prediction is:', rmspe(y_test, y_pred))

Starting training...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044954 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32633
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 128
[LightGBM] [Info] Start training from score 0.000177
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[79]	valid_0's l2: 3.0164e-09
Starting predicting...
The RMSPE of prediction is: 0.3790053938727757
